In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [2]:
a = tf.Variable(2.)
b = tf.Variable(3.)
with tf.GradientTape() as tape:
    c = a * b
gradient = tape.gradient(c, b)
b.assign_sub(gradient * .1)
print(gradient, b)

tf.Tensor(2.0, shape=(), dtype=float32) <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.8>


In [3]:
optimizer = tf.keras.optimizers.Adam(.2)
a = tf.Variable(2.)
b = tf.Variable(3.)
with tf.GradientTape() as tape:
    c = a * b
gradient = tape.gradient(c, b)
optimizer.apply_gradients([(gradient, b)])
print(gradient, b)

tf.Tensor(2.0, shape=(), dtype=float32) <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.7999945>


In [4]:
data = tf.keras.datasets.mnist.load_data()
(train_images, train_labels), (test_images, test_labels) = data
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

In [5]:
model = tf.keras.models.load_model('cnn.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

In [6]:
loss = tf.keras.losses.sparse_categorical_crossentropy
true = tf.reshape(1, (1, ))
pred = tf.reshape([.4, .3, .2], (1, 3))
print(true, pred, loss(true, pred))
true = tf.reshape(1, (1, ))
pred = tf.reshape([0., 1., 0.], (1, 3))
print(true, pred, loss(true, pred))

tf.Tensor([1], shape=(1,), dtype=int32) tf.Tensor([[0.4 0.3 0.2]], shape=(1, 3), dtype=float32) tf.Tensor([1.0986123], shape=(1,), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int32) tf.Tensor([[0. 1. 0.]], shape=(1, 3), dtype=float32) tf.Tensor([2.3841855e-07], shape=(1,), dtype=float32)


In [7]:
optimizer = tf.keras.optimizers.Adam()
loss_fun = tf.keras.losses.sparse_categorical_crossentropy
original_image = train_images[:1]
label = np.array([8])
original_prediction = model(original_image)

def train(steps):
    image = tf.Variable(np.array(original_image))
    for _ in range(steps):
        with tf.GradientTape() as tape:
            prediction = model(image)
            loss = loss_fun(label, prediction)
        gradient = tape.gradient(loss, image)
        optimizer.apply_gradients([(gradient, image)])
    return image

def play(steps):
    print(steps)
    image = train(steps)
    prediction = model(image)
    
    plt.figure(figsize=(18, 4))
    plt.subplot(1, 4, 1)
    plt.imshow(original_image[0])
    plt.subplot(1, 4, 2)
    plt.imshow(image[0])
    plt.subplot(1, 4, 3)
    plt.plot(np.arange(10), original_prediction[0])
    plt.subplot(1, 4, 4)
    plt.plot(np.arange(10), prediction[0])
    plt.show()
    return image
        
for steps in range(310, 320):
    image = play(steps)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]